# Debug Notebook

This notebook is used to debug and test the Spark environment and configurations.

## Setup

In [1]:
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.Row

import java.time.LocalDate
import java.time.format.DateTimeFormatter

Intitializing Scala interpreter ...

Spark Web UI available at http://DESKTOP-85RDGBL:4040
SparkContext available as 'sc' (version = 3.5.1, master = local[*], app id = local-1751571870816)
SparkSession available as 'spark'


import org.apache.spark.rdd.RDD
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.Row
import java.time.LocalDate
import java.time.format.DateTimeFormatter


In [4]:
val path_to_datasets = "../../../../datasets/"
val path_to_output = "../../../../src/main/outputs/"

val path_to_sample = path_to_datasets + "itineraries_sample.csv"

val spark = SparkSession.builder
  .appName("Debug")
  .getOrCreate()

path_to_datasets: String = ../../../../datasets/
path_to_output: String = ../../../../src/main/outputs/
path_to_sample: String = ../../../../datasets/itineraries_sample.csv
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3626603c


## Data loading, parsing, and cleaning

The structure of the dataset is as follows:
- **legId**: An identifier for the flight.
- **searchDate**: The date (`YYYY-MM-DD`) on which this entry was taken from Expedia.
- **flightDate**: The date (`YYYY-MM-DD`) of the flight.
- **startingAirport**: Three-character IATA airport code for the initial location.
- **destinationAirport**: Three-character IATA airport code for the arrival location.
- **fareBasisCode**: The fare basis code.
- **travelDuration**: The travel duration in hours and minutes.
- **elapsedDays**: The number of elapsed days (usually 0).
- **isBasicEconomy**: Boolean indicating whether the ticket is for basic economy.
- **isRefundable**: Boolean indicating whether the ticket is refundable.
- **isNonStop**: Boolean indicating whether the flight is non-stop.
- **baseFare**: The price of the ticket (in USD).
- **totalFare**: The price of the ticket (in USD) including taxes and other fees.
- **seatsRemaining**: Integer indicating the number of seats remaining.
- **totalTravelDistance**: The total travel distance in miles. This data is sometimes missing.
- **segmentsDepartureTimeEpochSeconds**: String containing the departure time (Unix time) for each leg of the trip, separated by `||`.
- **segmentsDepartureTimeRaw**: String containing the departure time (ISO 8601 format: `YYYY-MM-DDThh:mm:ss.000±[hh]:00`) for each leg of the trip, separated by `||`.
- **segmentsArrivalTimeEpochSeconds**: String containing the arrival time (Unix time) for each leg of the trip, separated by `||`.
- **segmentsArrivalTimeRaw**: String containing the arrival time (ISO 8601 format: `YYYY-MM-DDThh:mm:ss.000±[hh]:00`) for each leg of the trip, separated by `||`.
- **segmentsArrivalAirportCode**: String containing the IATA airport code for the arrival location for each leg of the trip, separated by `||`.
- **segmentsDepartureAirportCode**: String containing the IATA airport code for the departure location for each leg of the trip, separated by `||`.
- **segmentsAirlineName**: String containing the name of the airline that services each leg of the trip, separated by `||`.
- **segmentsAirlineCode**: String containing the two-letter airline code that services each leg of the trip, separated by `||`.
- **segmentsEquipmentDescription**: String containing the type of airplane used for each leg of the trip (e.g., "Airbus A321" or "Boeing 737-800"), separated by `||`.
- **segmentsDurationInSeconds**: String containing the duration of the flight (in seconds) for each leg of the trip, separated by `||`.
- **segmentsDistance**: String containing the distance traveled (in miles) for each leg of the trip, separated by `||`.
- **segmentsCabinCode**: String containing the cabin class for each leg of the trip (e.g., "coach"), separated by `||`.


In [5]:
case class Flight(
    legId: String,
    searchDate: LocalDate,
    flightDate: LocalDate,
    startingAirport: String,
    destinationAirport: String,
    fareBasisCode: String,
    travelDuration: Int,
    elapsedDays: Int,
    isBasicEconomy: Boolean,
    isRefundable: Boolean,
    isNonStop: Boolean,
    baseFare: Double,
    totalFare: Double,
    seatsRemaining: Int,
    totalTravelDistance: Double,
    segmentsDepartureTimeEpochSeconds: String,
    segmentsDepartureTimeRaw: String,
    segmentsArrivalTimeEpochSeconds: String,
    segmentsArrivalTimeRaw: String,
    segmentsArrivalAirportCode: String,
    segmentsDepartureAirportCode: String,
    segmentsAirlineName: String,
    segmentsAirlineCode: String,
    segmentsEquipmentDescription: String,
    segmentsDurationInSeconds: String,
    segmentsDistance: String,
    segmentsCabinCode: String
)

defined class Flight


Now we define the parsing function.
> Because the `travelDuration` rows are in ISO 8601 format, we will convert them to integer minutes for easier processing.

In [6]:
def parseLine(line: String): Flight = {
    val cols = line.split(",", -1).map(_.trim)

    val dateFormatter = DateTimeFormatter.ofPattern("yyyy-MM-dd")

    def parseDurationToMinutes(duration: String): Int = {
        val hourPattern = "PT(\\d+)H".r
        val minutePattern = "PT(?:\\d+H)?(\\d+)M".r

        val hours = hourPattern.findFirstMatchIn(duration).map(_.group(1).toInt).getOrElse(0)
        val minutes = minutePattern.findFirstMatchIn(duration).map(_.group(1).toInt).getOrElse(0)

        hours * 60 + minutes
    }

    def toIntSafe(s: String): Int = if (s.isEmpty) 0 else s.toInt
    def toDoubleSafe(s: String): Double = if (s.isEmpty) 0.0 else s.toDouble
    def toBooleanSafe(s: String): Boolean = if (s.isEmpty) false else s.toBoolean

    Flight(
        legId = cols(0),
        searchDate = LocalDate.parse(cols(1), dateFormatter),
        flightDate = LocalDate.parse(cols(2), dateFormatter),
        startingAirport = cols(3),
        destinationAirport = cols(4),
        fareBasisCode = cols(5),
        travelDuration = parseDurationToMinutes(cols(6)),
        elapsedDays = toIntSafe(cols(7)),
        isBasicEconomy = toBooleanSafe(cols(8)),
        isRefundable = toBooleanSafe(cols(9)),
        isNonStop = toBooleanSafe(cols(10)),
        baseFare = toDoubleSafe(cols(11)),
        totalFare = toDoubleSafe(cols(12)),
        seatsRemaining = toIntSafe(cols(13)),
        totalTravelDistance = toDoubleSafe(cols(14)),
        segmentsDepartureTimeEpochSeconds = cols(15),
        segmentsDepartureTimeRaw = cols(16),
        segmentsArrivalTimeEpochSeconds = cols(17),
        segmentsArrivalTimeRaw = cols(18),
        segmentsArrivalAirportCode = cols(19),
        segmentsDepartureAirportCode = cols(20),
        segmentsAirlineName = cols(21),
        segmentsAirlineCode = cols(22),
        segmentsEquipmentDescription = cols(23),
        segmentsDurationInSeconds = cols(24),
        segmentsDistance = cols(25),
        segmentsCabinCode = cols(26)
    )
}

parseLine: (line: String)Flight


Now we can load the dataset and parse it into an RDD of `Flight` case class.

In [8]:
var rddRaw = spark.sparkContext.textFile(path_to_sample).filter(line => !line.startsWith("legId"))
var rddParsed = rddRaw.map(parseLine)

rddRaw: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[2] at filter at <console>:31
rddParsed: org.apache.spark.rdd.RDD[Flight] = MapPartitionsRDD[3] at map at <console>:32


In [20]:
val sampleRaw = rddRaw.take(5)              // Always safe: text only
val sampleParsed = sampleRaw.map(parseLine) // Run locally, no Spark involved
sampleParsed.foreach(println)

Flight(cb345693479e84838dfaac346b8e8d6a,2022-04-16,2022-04-17,ATL,BOS,VNR,339,0,false,false,false,183.0,290.58,0,0.0,1650187560||1650196800,2022-04-17T05:26:00.000-04:00||2022-04-17T08:00:00.000-04:00,1650194400||1650207900,2022-04-17T07:20:00.000-04:00||2022-04-17T11:05:00.000-04:00,FLL||BOS,ATL||FLL,Spirit Airlines||Spirit Airlines,NK||NK,AIRBUS INDUSTRIE A321 SHARKLETS||,6840||11100,None||None,coach||coach)
Flight(80d7a9d00fbcdd1aaa9ec61187b5dfe1,2022-04-16,2022-04-17,ATL,DFW,YNR,757,0,false,false,false,232.0,339.58,0,0.0,1650195000||1650228900,2022-04-17T07:30:00.000-04:00||2022-04-17T16:55:00.000-04:00,1650201840||1650240420,2022-04-17T09:24:00.000-04:00||2022-04-17T19:07:00.000-05:00,FLL||DFW,ATL||FLL,Spirit Airlines||Spirit Airlines,NK||NK,AIRBUS INDUSTRIE A321 SHARKLETS||AIRBUS INDUSTRIE A321 SHARKLETS,6840||11520,None||None,coach||coach)
Flight(ce824fe4eea70d2310b0f01559b52575,2022-04-16,2022-04-17,ATL,DTW,QAA0OFEN,344,0,false,false,false,255.81,298.6,1,840.0,1650232380||16502

sampleRaw: Array[String] = Array(cb345693479e84838dfaac346b8e8d6a,2022-04-16,2022-04-17,ATL,BOS,VNR,PT5H39M,0,False,False,False,183.0,290.58,0,,1650187560||1650196800,2022-04-17T05:26:00.000-04:00||2022-04-17T08:00:00.000-04:00,1650194400||1650207900,2022-04-17T07:20:00.000-04:00||2022-04-17T11:05:00.000-04:00,FLL||BOS,ATL||FLL,Spirit Airlines||Spirit Airlines,NK||NK,AIRBUS INDUSTRIE A321 SHARKLETS||,6840||11100,None||None,coach||coach, 80d7a9d00fbcdd1aaa9ec61187b5dfe1,2022-04-16,2022-04-17,ATL,DFW,YNR,PT12H37M,0,False,False,False,232.0,339.58,0,,1650195000||1650228900,2022-04-17T07:30:00.000-04:00||2022-04-17T16:55:00.000-04:00,1650201840||1650240420,2022-04-17T09:24:00.000-04:00||2022-04-17T19:07:00.000-05:00,FLL||DFW,ATL||FLL,Spirit Airlines||Spirit Airlines,NK||NK,AIRBUS INDUSTRIE A...


Before proceding, we will clean the data by removing records with missing or invalid values.

In [ ]:
val rddClean = rddParsed.filter { row =>
    (0 until row.length).forall { i =>
        val value = row.get(i)
        value != null && value.toString.nonEmpty
    }
}

In [ ]:
val totalRows = rddClean.count()
println(s"Total rows: $totalRows")

The other columns are already in a suitable format, so we can proceed with the analysis. Other columns will be left as strings because they are out of scope for the future jobs.

## Jobs

- **Average Flight Price per Route:** Calculate the average ticket price for each departure-destination pair.
- **Direct vs. Connecting Flights Price Comparison:** Compare average prices between non-stop and connecting flights.

### Job 1: Average Flight Price per Route

#### Non-optimized version

#### Optimized version


### Job 2: Direct vs. Connecting Flights Price Comparison

#### Non-optimized version

#### Optimized version